# Day 06 — RAG basics (lightweight)

This notebook shows a **simple RAG loop**: retrieve notes, then answer grounded on the retrieved context.


In [ ]:
notes = [
    {"title": "Prompt chaining", "text": "Chain prompts by using a planner, executor, and critic."},
    {"title": "Tool routing", "text": "Route user requests to specialized tools based on intent."},
    {"title": "Memory", "text": "Keep a short preference list and inject it into prompts."},
]

def retrieve(query, docs, top_k=2):
    scored = []
    query_terms = set(query.lower().split())
    for doc in docs:
        doc_terms = set(doc["text"].lower().split())
        score = len(query_terms & doc_terms)
        scored.append((score, doc))
    ranked = sorted(scored, key=lambda x: x[0], reverse=True)
    return [doc for score, doc in ranked[:top_k] if score > 0]


In [ ]:
question = "How do I keep a user's preferences available for future prompts?"
context_docs = retrieve(question, notes)
context = "\n\n".join(f"- {d['title']}: {d['text']}" for d in context_docs)
context


Use the prompt template in `prompts/rag_prompt.txt` to send `context` + `question` to your model of choice.
